In [62]:
import os
import ffmpeg
from datetime import timedelta
import yaml
import pandas as pd
import numpy as np
from itertools import combinations
from itertools import product

In [63]:
df = pd.read_csv('D:\\GITHUB\\Data_CEP\\Exploration\\dataframe_expanded.csv')

In [64]:
df_cross = df[df['cross'] == 1] #length = 8636

In [65]:
print(len(df_cross[df_cross['flank'] == 'center']))

400


In [68]:
#Flank: 
print(df_cross['flank'].unique())
df_cross_non_flank = df_cross[df_cross['flank'].isna()]
#df_cross_non_flank
print(len(df_cross[df_cross['flank'].isna()]))
df_cross_all_flank = df_cross[~df_cross['flank'].isna()]
print(len(df_cross_all_flank))

df_cross['flank_right'] = 0
df_cross['flank_left'] = 0
df_cross['flank_center'] = 0
df_cross.loc[(df_cross['flank'] == 'right'), 'flank_right'] = 1
df_cross.loc[(df_cross['flank'] == 'left'), 'flank_left'] = 1
df_cross.loc[(df_cross['flank'] == 'center'), 'flank_center'] = 1

['right' 'left' 'center']
0
7895


In [69]:
print(len(df_cross[df_cross['flank_center'] == 1]))

400


In [70]:
#Direction of the pass:
#Here is shown that when there is no direction given, the crosses have a length longer than 12 meters, which according to the wyscout glossary is not possible. These rows are omitted from the dataset to sample from.
df_cross_non_direction = df_cross[(df_cross['forward_pass'] == 0) & (df_cross['back_pass'] == 0) & (df_cross['lateral_pass'] == 0)]
print(len(df_cross_non_direction))
df_cross_non_direction_more_than_12 = df_cross_non_direction[df_cross_non_direction['pass_length'] > 12]
print(len(df_cross_non_direction_more_than_12))
df_cross_lateral_less_than_12 = df_cross[(df_cross['lateral_pass'] == 1) & (df_cross['pass_length'] < 12)]
print(len(df_cross_lateral_less_than_12))
df_cross_back_less_than_12 = df_cross[(df_cross['back_pass'] == 1) & (df_cross['pass_length'] < 12)]
print(len(df_cross_back_less_than_12))
df_cross_forward_less_than_12 = df_cross[(df_cross['forward_pass'] == 1) & (df_cross['pass_length'] < 12)]
print(len(df_cross_forward_less_than_12))

1620
1216
0
29
27


In [71]:
df_cross_non_direction

,Unnamed: 0,event_id,match_id,match_period,minute,second,match_timestamp,video_timestamp,primary_type,secondary_types,...,shot_after_free_kick,acceleration,penalty_foul,penalty_goal,penalty_conceded_goal,third_assist,penalty_save,flank_right,flank_left,flank_center
515,515,1723218895,5476240,1H,31,1,00:31:01.677,1862.677617,pass,"cross, cross_blocked",...,0,0,0,0,0,0,0,1,0,0
617,617,1723218835,5476240,1H,35,35,00:35:35.638,2136.638387,pass,"cross, cross_blocked",...,0,0,0,0,0,0,0,0,1,0
2144,2144,1720901061,5476241,1H,20,35,00:20:35.907,1236.907344,pass,"cross, cross_blocked",...,0,0,0,0,0,0,0,0,1,0
2413,2413,1720901331,5476241,1H,40,47,00:40:47.055,2448.055109,pass,"cross, cross_blocked, touch_in_box",...,0,0,0,0,0,0,0,1,0,0
2573,2573,1720901626,5476241,2H,47,34,00:47:34.255,3047.255392,pass,"cross, cross_blocked",...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494103,494103,2139887999,5476537,2H,84,34,01:24:34.186,5207.186517,pass,"cross, deep_completed_cross, touch_in_box",...,0,0,0,0,0,0,0,1,0,0
494885,494885,2140029111,5476539,1H,32,45,00:32:45.399,1966.399958,pass,"cross, loss",...,0,0,0,0,0,0,0,1,0,0
495110,495110,2140029305,5476539,1H,44,20,00:44:20.124,2661.124997,pass,"cross, deep_completed_cross, key_pass, shot_as...",...,0,0,0,0,0,0,0,1,0,0
495458,495458,2140029645,5476539,2H,62,32,01:02:32.859,3783.859502,pass,"cross, cross_blocked",...,0,0,0,0,0,0,0,0,1,0


In [73]:
#Dividing the smaller than 12 meter passes over the forward, back and lateral based on their passing degree. 
df_cross.loc[
    (df_cross['forward_pass'] == 0) & 
    (df_cross['back_pass'] == 0) & 
    (df_cross['lateral_pass'] == 0) & 
    (((df_cross['pass_angle'] > -135) & (df_cross['pass_angle'] < -45)) | 
     ((df_cross['pass_angle'] > 45) & (df_cross['pass_angle'] < 135))),
    'lateral_pass'
] = 1

df_cross.loc[
    (df_cross['forward_pass'] == 0) & 
    (df_cross['back_pass'] == 0) & 
    (df_cross['lateral_pass'] == 0) & 
    (df_cross['pass_angle'] >= -45) & 
    (df_cross['pass_angle'] <= 45), 
    'forward_pass'
] = 1

df_cross.loc[
    (df_cross['forward_pass'] == 0) & 
    (df_cross['back_pass'] == 0) & 
    (df_cross['lateral_pass'] == 0) & 
    ((df_cross['pass_angle'] <= -135) | 
     (df_cross['pass_angle'] >= 135)), 
    'back_pass'
] = 1

In [74]:
#Precision/Accuracy/Successfulness of the pass:
print(df_cross['pass_accurate'].unique())
#Only true or false so that is perfect

[True False]


In [75]:
df_cross['accurate'] = 0
df_cross['not_accurate'] = 0
# Set the values based on the 'pass_accurate' column
# Set values based on 'pass_accurate' column
df_cross.loc[(df_cross['pass_accurate'] == True), 'accurate'] = 1
df_cross.loc[(df_cross['pass_accurate'] == False), 'not_accurate'] = 1

In [76]:
def select_minimum_clips(df, primary_type, attribute_sets, max_difference_per_set=500, min_clips_per_attribute=1050, chunk_size=10000):
    # Filter the DataFrame for the specified primary type
    primary_df = df 
    # Dictionary to hold the selected clips for each attribute set
    selected_clips_dict = {attr_set: [] for attr_set in attribute_sets}
    
    # Function to check if all attribute sets have enough clips
    def all_attribute_sets_satisfied():
        return all(len(clips) >= min_clips_per_attribute * len(attr_set) for attr_set, clips in selected_clips_dict.items())
    
    # Iterate through the DataFrame once per attribute set
    for attr_set in attribute_sets:
        attr_clips = []
        for attr in attr_set:
            # Extract clips where the specific column (attr) is True (1)
            attr_clips.extend(primary_df[primary_df[attr] == 1]['event_id'].tolist())
        
        # Shuffle the clips to randomize selection
        np.random.shuffle(attr_clips)
        
        # Select exactly min_clips_per_attribute clips for this attribute set
        selected_clips_dict[attr_set] = attr_clips[:min_clips_per_attribute]
        
        # Check if all attribute sets have enough clips after each set
        if all_attribute_sets_satisfied():
            break
    
    # Combine all selected clips ensuring no duplicates
    selected_clips = set()
    for clips in selected_clips_dict.values():
        selected_clips.update(clips)

    return selected_clips

In [77]:
# Shuffle the rows of the DataFrame
df_shuffled = df_cross.sample(frac=1).reset_index(drop=True)

# Define primary type and attribute sets as tuples
# Define primary type and attribute sets as tuples
primary_type = 'cross'
attribute_sets = [
    ('flank_right', 'flank_center', 'flank_left'),
    ('forward_pass', 'back_pass', 'lateral_pass'),
    ('accurate', 'not_accurate'),
]

# Select minimum clips to cover all classifications
selected_clips = select_minimum_clips(df_shuffled, primary_type, attribute_sets)
print(f"Selected clip IDs: {selected_clips}")
print(f"Number of selected clips: {len(selected_clips)}")

Selected clip IDs: {1720901633, 1773518853, 1789812753, 1772937233, 1874329619, 1771814929, 2092408859, 2099060763, 1866522653, 2103984158, 1777958944, 2126766115, 2121408550, 1874329643, 2099945517, 2099060783, 1772937267, 2125348920, 2108342333, 2140028992, 2088097462, 1744224326, 2102878285, 2099945550, 1813545039, 2126766159, 1821614166, 2126766168, 2102878303, 1871880288, 1720901733, 2133966955, 1797029996, 2121408623, 2121408624, 1866522738, 1770709106, 2126766200, 2105262203, 1850122367, 1764769922, 2105262215, 1720901772, 1770709134, 1797030032, 2133966994, 1785888916, 1819115678, 2093875359, 1819115681, 1874452643, 1884577958, 1810759849, 1742405804, 2130903214, 1773519032, 2125349055, 1937457346, 1773519045, 1819115721, 1744060623, 1744273618, 1747542246, 1789812972, 2140029165, 1886314734, 1789812978, 1768202489, 1850122490, 1827938555, 1747542272, 1741177091, 2100199683, 1744273671, 1724481804, 2133967130, 2099945754, 1937457438, 1819115807, 1744060708, 2126766373, 21253491

In [ ]:
df_selected_clips = df_shuffled[df_shuffled['event_id'].isin(selected_clips)].reset_index(drop=True)


In [79]:
df_selected_clips.to_csv('D:\\GITHUB\\Data_CEP\\Exploration\\selected_clips_cross.csv', index=False)

In [78]:
list = ['flank_right', 'flank_center', 'flank_left', 'forward_pass', 'back_pass', 'lateral_pass', 'accurate', 'not_accurate']
for att in list:
    print(len(df_selected_clips[df_selected_clips[att] ==1]))

1291
140
1328
142
299
2318
1059
1700
